In [1]:
import os, numpy as np
import pandas as pd

os.environ["KERAS_BACKEND"] = "tensorflow"

import keras 
from sklearn import datasets

from sklearn.model_selection import train_test_split

from keras.utils import np_utils



Using TensorFlow backend.


# Build a Keras Model

In [2]:


def create_model():
    from keras.models import Sequential
    from keras.layers.core import Dense, Activation

    model = Sequential()
    model.add(Dense(12, input_dim=13))
    model.add(Dense(1))
    model.compile(optimizer='rmsprop', loss='mse')
    return model

In [3]:
boston  = datasets.load_boston()
train_X, test_X, train_y, test_y = train_test_split(boston.data, boston.target, train_size=0.8, test_size=0.2, random_state=1960)


In [4]:

from keras.wrappers.scikit_learn import KerasRegressor

clf = KerasRegressor(build_fn=create_model, epochs=12, verbose=0)

print(train_X.shape , train_y.shape)
clf.fit(train_X, train_y, verbose=0)

(404, 13) (404,)


In [5]:
print(test_X.shape)
preds = clf.predict(test_X[0,:].reshape(1,13))
print(preds)


(102, 13)
19.344752418500537


# Generate SQL Code from the Model

In [6]:
import json, requests, base64, dill as pickle, sys



sys.setrecursionlimit(200000)
pickle.settings['recurse'] = False

# no luck for the web service... pickling feature of tensorflow and/or keras objects seems not to be a priority.
# there is a lot of github issues in the two projects when I search for pickle keyword!!!.

def test_ws_sql_gen(pickle_data):
    WS_URL="http://localhost:1888/model"
    b64_data = base64.b64encode(pickle_data).decode('utf-8')
    data={"Name":"model1", "PickleData":b64_data , "SQLDialect":"postgresql"}
    r = requests.post(WS_URL, json=data)
    print(r.__dict__)
    content = r.json()
    # print(content)
    lSQL = content["model"]["SQLGenrationResult"][0]["SQL"]
    return lSQL;



def test_sql_gen(keras_regressor , metadata):
    import sklearn2sql.PyCodeGenerator as codegen
    cg1 = codegen.cAbstractCodeGenerator();
    cg1.mOptions.mActivateMaterialization = False
    lSQL = cg1.generateCodeWithMetadata(clf, metadata, dsn = None, dialect = "postgresql");
    return lSQL[1]


In [7]:
# commented .. see above
# pickle_data = pickle.dumps(clf)
# lSQL = test_ws_sql_gen(pickle_data)
# print(lSQL[0:2000])

In [8]:
lMetaData = {}
lMetaData['features'] = list(boston.feature_names);
lMetaData["targets"] = ['TGT']
lMetaData['primary_key'] = 'KEY'
lMetaData['table'] = 'boston'

    
lSQL = test_sql_gen(clf , lMetaData)


cGenerationWrapperFactory::createWrapper() <class 'keras.wrappers.scikit_learn.KerasRegressor'>
BACKEND_DIALECT postgresql
CREATING_DATABASE_BACKEND_DSN_DIALECT 1.2.7 None postgresql
KERAS_REG_GENERATE_EXPRESSION_START <class 'sqlalchemy.sql.selectable.Alias'> <class 'sklearn2sql.GenericModels.Keras.cSerializedKeras'>
{'mKerasData': <keras.wrappers.scikit_learn.KerasRegressor object at 0x7f4f77292fd0>, 'mFeatureNames': ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT'], 'mClasses': None, 'mMode': 'Regression'}
{'build_fn': <function create_model at 0x7f4f772a1d90>, 'sk_params': {'epochs': 12, 'verbose': 0}, 'model': <keras.engine.sequential.Sequential object at 0x7f4f77292f98>}
GENERATING_LAYER 0 dense_1 <class 'keras.layers.core.Dense'> {'input_spec': InputSpec(min_ndim=2, axes={-1: 13}), 'supports_masking': True, 'stateful': False, '_trainable_weights': [<tf.Variable 'dense_1/kernel:0' shape=(13, 12) dtype=float64_ref>, <tf.Variable 'den

In [9]:
print(lSQL)

WITH keras_input AS 
(SELECT "ADS"."KEY" AS "KEY", "ADS"."CRIM" AS "CRIM", "ADS"."ZN" AS "ZN", "ADS"."INDUS" AS "INDUS", "ADS"."CHAS" AS "CHAS", "ADS"."NOX" AS "NOX", "ADS"."RM" AS "RM", "ADS"."AGE" AS "AGE", "ADS"."DIS" AS "DIS", "ADS"."RAD" AS "RAD", "ADS"."TAX" AS "TAX", "ADS"."PTRATIO" AS "PTRATIO", "ADS"."B" AS "B", "ADS"."LSTAT" AS "LSTAT" 
FROM boston AS "ADS"), 
keras_input_1 AS 
(SELECT keras_input."KEY" AS "KEY", keras_input."CRIM" AS "CRIM", keras_input."ZN" AS "ZN", keras_input."INDUS" AS "INDUS", keras_input."CHAS" AS "CHAS", keras_input."NOX" AS "NOX", keras_input."RM" AS "RM", keras_input."AGE" AS "AGE", keras_input."DIS" AS "DIS", keras_input."RAD" AS "RAD", keras_input."TAX" AS "TAX", keras_input."PTRATIO" AS "PTRATIO", keras_input."B" AS "B", keras_input."LSTAT" AS "LSTAT" 
FROM keras_input), 
layer_dense_1 AS 
(SELECT keras_input_1."KEY" AS "KEY", -0.0246028715095858 + 0.06550673902803521 * keras_input_1."CRIM" + -0.036804466805360835 * keras_input_1."ZN" + -0.265491

# Execute the SQL Code

In [10]:
# save the dataset in a database table


import sqlalchemy as sa

#engine = sa.create_engine('sqlite://' , echo=False)
engine = sa.create_engine("postgresql://db:db@localhost/db?port=5432", echo=False)
conn = engine.connect()

lTable = pd.DataFrame(boston.data);
lTable.columns = lMetaData['features']
lTable['TGT'] = boston.target
lTable['KEY'] = range(boston.data.shape[0])
lTable.to_sql(lMetaData['table'] , conn,   if_exists='replace', index=False)

In [11]:
sql_output = pd.read_sql(lSQL , conn);
conn.close()

In [12]:
sql_output.sample(12, random_state=1960)

,KEY,Estimator
230,230,19.344752
112,112,15.972185
125,125,20.570982
9,9,22.340942
213,213,19.095113
109,109,17.522685
127,127,14.947902
244,244,22.793336
406,406,25.237959
490,490,3.708052


# Keras Prediction

In [13]:
skl_output = pd.DataFrame()
skl_output_key = pd.DataFrame(list(range(boston.data.shape[0])), columns=['KEY']);
skl_output_estimator = pd.DataFrame(clf.predict(boston.data), columns=['Estimator'])
skl_output = skl_output_key
skl_output['Estimator'] = skl_output_estimator
skl_output.sample(12, random_state=1960)

,KEY,Estimator
230,230,19.344752
112,112,15.972185
125,125,20.570982
9,9,22.340942
213,213,19.095113
109,109,17.522685
127,127,14.947902
244,244,22.793336
406,406,25.237959
490,490,3.708052


# Comparing the SQL and Keras Predictions

In [14]:
sql_skl_join = skl_output.join(sql_output , how='left', on='KEY', lsuffix='_skl', rsuffix='_sql')
sql_skl_join['Error'] = sql_skl_join.Estimator_sql - sql_skl_join.Estimator_skl

In [15]:
sql_skl_join.head(12)

,KEY_skl,Estimator_skl,KEY_sql,Estimator_sql,Error
0,0,20.918060,0,20.918060,-4.263256e-14
1,1,22.171661,1,22.171661,1.421085e-14
2,2,21.459476,2,21.459476,9.592327e-14
3,3,22.897451,3,22.897451,1.421085e-14
4,4,23.343215,4,23.343215,-1.421085e-14
5,5,23.168069,5,23.168069,3.907985e-14
6,6,21.425313,6,21.425313,-2.842171e-14
7,7,22.737048,7,22.737048,1.065814e-13
8,8,22.816067,8,22.816067,1.207923e-13
9,9,22.340942,9,22.340942,5.329071e-14


In [16]:
sql_skl_join.describe()

,KEY_skl,Estimator_skl,KEY_sql,Estimator_sql,Error
count,506.000000,506.000000,506.000000,506.000000,5.060000e+02
mean,252.500000,18.781477,252.500000,18.781477,1.889988e-14
std,146.213884,7.474813,146.213884,7.474813,4.939573e-14
min,0.000000,-6.690646,0.000000,-6.690646,-1.811884e-13
25%,126.250000,15.685156,126.250000,15.685156,-1.421085e-14
50%,252.500000,20.030195,252.500000,20.030195,1.776357e-14
75%,378.750000,22.419377,378.750000,22.419377,5.140333e-14
max,505.000000,80.034728,505.000000,80.034728,1.527667e-13
